# Работа с SQL базой в pandas

Для работы с базой через pandas требуется предварительно настроить движок на основе библиотеки SQLAlchemy.

В этом задании также будет рассмотрена работа с mysql базой sakila.

In [2]:
import pandas as pd
import sqlalchemy as sa

user='admin'
password='admin'
host='mysql'
port=3306
database='sakila'

sql_dialect="mysql"
sql_driver="mysqlconnector"

try:
    engine = sa.create_engine(f"{sql_dialect}+{sql_driver}://{user}:{password}@{host}:{port}/{database}")    
    with engine.connect() as cnx:
        result = cnx.execute(sa.text("SELECT 1"))
        print("SQLAlchemy engine works properly")
except Exception as e:
    print(f"Error connecting to database: {e}")

SQLAlchemy engine works properly


Когда движок настроен можно загружать из базы таблицы целиком в виде DataFrame

In [3]:
table_name = "city"
df = pd.read_sql_table(table_name, engine)
df.head()

,city_id,city,country_id,last_update
0,1,A Coruña (La Coruña),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25
3,4,Acuña,60,2006-02-15 04:45:25
4,5,Adana,97,2006-02-15 04:45:25


Также можно выполнять SQL запросы и загружать их результаты в виде DataFrame

In [4]:
qu = "SELECT * FROM city"
df = pd.read_sql_query(qu, engine)
df.head()

,city_id,city,country_id,last_update
0,1,A Coruña (La Coruña),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25
3,4,Acuña,60,2006-02-15 04:45:25
4,5,Adana,97,2006-02-15 04:45:25


## Задание 1

Постройте SQL запрос, который возвращает список названий фильмов и языков, на которых он сняты. Далее выполните этот запрос при помощи pandas и исползуя полученный набора данных подсчитайте, сколько фильмов на каждом из языков имеются в базе.

In [14]:
# Ваш код:
qu = """select f.film_id, f.title, l.name
from film f
join `language` l on l.language_id = f.language_id;"""

In [15]:
films_and_languages = pd.read_sql_query(qu, engine)
films_and_languages

,film_id,title,name
0,1,ACADEMY DINOSAUR,English
1,2,ACE GOLDFINGER,English
2,3,ADAPTATION HOLES,English
3,4,AFFAIR PREJUDICE,English
4,5,AFRICAN EGG,English
...,...,...,...
995,996,YOUNG LANGUAGE,English
996,997,YOUTH KICK,English
997,998,ZHIVAGO CORE,English
998,999,ZOOLANDER FICTION,English


In [21]:
films_and_languages.groupby("name").count()

,film_id,title
name,,
English,1000,1000


## Задание 2

Загрузите таблицы 'film_category' и 'category' в pandas и удалите из полученных наборов данных столбцы 'last_update'. Изучите документацию метода `.merge()` и выполните объединение считанных наборов данных. Используя полученный набор данных подсчитайте количество фильмов каждой категории.

In [22]:
# Ваш код:
qu = """
select * from film_category;
"""
film_category = pd.read_sql_query(qu, engine)

In [23]:
qu = """
select * from category;
"""
category = pd.read_sql_query(qu, engine)

In [28]:
film_category = film_category.drop("last_update", axis=1)
category = category.drop("last_update", axis=1)

In [32]:
category.merge(film_category).groupby("name").count()

,category_id,film_id
name,,
Action,64,64
Animation,66,66
Children,60,60
Classics,57,57
Comedy,58,58
Documentary,68,68
Drama,62,62
Family,69,69
Foreign,73,73


## Задание 3

Постройте запрос, который возвращает два столбца — 'person' (объединённые имя и фамилия клиента) и 'title' (название фильма, который клиент брал в прокате). 
Загрузите результат этого запроса в виде набора данных pandas. Далее, на основе него, в pandas постройте новый набор данных в котором в первом столбце стояли бы имя и фамилия клиента, а во втором — текстовая строка, в которой перечислены все фильмы, которые он брал. Для этого, вероятно, потребуется написать собственную функцию агрегации. Предусмотрите удаление повторов — название каждого фильма должно появляться только один раз. Фильмы должны быть отсортированы по алфавиту.

In [72]:
# Ваш код:
qu = """
select concat(c.first_name, ' ', c.last_name) as person, f.title 
from rental r
join customer c on c.customer_id = r.customer_id 
join inventory i on i.inventory_id = r.inventory_id 
join film f on f.film_id = i.film_id;
"""
persons_and_titles = pd.read_sql_query(qu, engine)

In [73]:
def all_films(x):
    return ', '.join(sorted(x.unique()))

In [74]:
answer3 = persons_and_titles.groupby("person").agg(all_films)

In [75]:
answer3

,title
person,
AARON SELBY,"ARACHNOPHOBIA ROLLERCOASTER, BEAUTY GREASE, CO..."
ADAM GOOCH,"ALADDIN CALENDAR, BAREFOOT MANCHURIAN, COMMAND..."
ADRIAN CLARY,"BOOGIE AMELIE, BOUND CHEAPER, CHICAGO NORTH, D..."
AGNES BISHOP,"ANACONDA CONFESSIONS, ATLANTIS CAUSE, CAMELOT ..."
ALAN KAHN,"AGENT TRUMAN, ALIEN CENTER, BANGER PINOCCHIO, ..."
...,...
WILLIE MARKHAM,"ACADEMY DINOSAUR, ALONE TRIP, BANGER PINOCCHIO..."
WILMA RICHARDS,"BACKLASH UNDEFEATED, BARBARELLA STREETCAR, CAN..."
YOLANDA WEAVER,"BLUES INSTINCT, CANDIDATE PERDITION, CASABLANC..."


In [76]:
answer3 = answer3.reset_index()

In [77]:
answer3[answer3["person"] == "AARON SELBY"]["title"].item()

'ARACHNOPHOBIA ROLLERCOASTER, BEAUTY GREASE, CORE SUIT, DORADO NOTTING, DRIFTER COMMANDMENTS, DRUMLINE CYCLONE, EXPECATIONS NATURAL, FELLOWSHIP AUTUMN, FEVER EMPIRE, LIAISONS SWEET, MUSCLE BRIGHT, NETWORK PEAK, ORANGE GRAPES, PATIENT SISTER, PERFECT GROOVE, SHOCK CABIN, SLEEPING SUSPECTS, SWEDEN SHINING, TEEN APOLLO, USUAL UNTOUCHABLES, VALENTINE VANISHING, WILLOW TRACY, ZHIVAGO CORE'

## Задание 4

Сначала посредством SQL и pandas получите набор данных, который демонстрирует результативность сотрудников проката. Затем выберите самого результативного и получите набор данных, в котором в качестве индекса задано название фильма и имеется единственная колонка с описанием фильма. Вероятно, для этого потребуется использовать метод `.set_index()`.

In [81]:
# Ваш код:
qu = """
select f.first_name, f.last_name, COUNT(r.rental_id) as sale_count
from staff f
join rental r on r.staff_id = f.staff_id 
group by r.staff_id;
"""

In [82]:
staff = pd.read_sql_query(qu, engine)

In [83]:
staff

,first_name,last_name,sale_count
0,Mike,Hillyer,8040
1,Jon,Stephens,8004


In [85]:
qu = """
select title, description
from film
"""

In [86]:
films = pd.read_sql_query(qu, engine)

In [87]:
films.set_index("title")

,description
title,
ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...
ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...
ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...
AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...
AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...
...,...
YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...
YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...
ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...
